# Grid Search for K-Means in Latent Space (+ Comparison to Initial Space by projection)

In [1]:
import os

from multiprocessing.pool import ThreadPool

import ctypes
from ctypes import *

import numpy as np

from tensorflow.keras.models import load_model

from autoencoder import Autoencoder

from helper_funcs import *

import pandas
pandas.set_option('display.max_rows', None)

from params import get_kmeans_eval_object, get_centroids, convert_to_2d_array, get_stotal, free_centroids, free_kmeans

2024-01-07 10:29:35.877051: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-07 10:29:35.945931: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-07 10:29:35.947512: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-07 10:29:37.008478: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
models = os.listdir('./models/')

dataset = b'MNIST/input.dat'
query   = b'MNIST/query.dat'

model_to_files = {}
for i, model in enumerate(models):
    normalized_dataset = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_dataset.dat'
    normalized_query   = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_query.dat'
    encoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_dataset.dat'
    encoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_query.dat'

    model_to_files.update({models[i] : [normalized_dataset, normalized_query,
                                        encoded_dataset, encoded_query]})

n = 60000

In [3]:
for model in model_to_files:
    normalized_dataset, normalized_query, encoded_dataset, encoded_query = model_to_files[model]

    model = b'models/' + model.encode()

    # load model
    autoencoder = load_model(model.decode())
    shape = autoencoder.layers[-2].output_shape[1:] # get shape of encoded layer

    # load dataset
    x_train = load_dataset(dataset)
    x_train = x_train.astype('float32') / 255.
    x_test = load_dataset(query)
    x_test = x_test.astype('float32') / 255.
    if len(shape) == 3: # if model type is convolutional
        x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
        x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
    else:
        x_train = np.reshape(x_train, (len(x_train), 784))
        x_test = np.reshape(x_test, (len(x_test), 784))

    encoded_train = autoencoder.encode(x_train)
    encoded_test = autoencoder.encode(x_test)

    # deflatten encoded datasets
    encoded_train = deflatten_encoded(encoded_train, shape)
    encoded_test = deflatten_encoded(encoded_test, shape)

    # save original datasets normalized
    save_decoded_binary(x_train, normalized_dataset)
    save_decoded_binary(x_test, normalized_query)

    # normalize encoded datasets
    encoded_train = normalize(encoded_train)
    encoded_test = normalize(encoded_test)

    # save encoded datasets
    save_encoded_binary(encoded_train, encoded_dataset)
    save_encoded_binary(encoded_test, encoded_query)

313/313 [==============================] - 1s 2ms/step


In [4]:
best_params_lsh = [4, 7, 1, 0.6]            # L, k, limit_queries, window
best_params_hypercube = [67, 3, 1000, 0.42] # M, k, probes, window

rows = []

In [5]:
def print_results(stotal_lat_init, silouette_lat_init, obj_func):
    print("Total silhouette:", stotal_lat_init)
    print("Silhouette per cluster:", silouette_lat_init)
    print("Objective function:", obj_func)
    print("-------------------------")

def decode_centroids(model, centroids): # decodes centroids and returns them as c array
    model = b'models/' + model.encode()

    autoencoder = load_model(model.decode())
    shape = autoencoder.layers[-2].output_shape[1:] # shape of encoded layer

    centroids = deflatten_encoded(centroids, shape)
    decoded_centroids = autoencoder.decode(centroids)
    decoded_centroids = flatten_encoded(decoded_centroids)

    decoded_centroids = decoded_centroids.astype(np.float64) # cast to float64, equivalent to double in c
    decoded_centroids = decoded_centroids.flatten() # flatten to 1d numpy array
    decoded_centroids = decoded_centroids.ctypes.data_as(ctypes.POINTER(ctypes.c_double)) # convert to c array
    decoded_centroids = convert_to_2d_array(decoded_centroids, 784) # convert to 2d c array, shape: (10, 784)

    return decoded_centroids

def compute_silhouette(model, config):
    kmeans = get_kmeans_eval_object(conf=config)
    centroids, _ = get_centroids(kmeans)

    decoded_centroids = decode_centroids(model, centroids)

    # total silhouette and silhouette per cluster in latent space converted to initial space
    stotal_lat_init, silouette_lat_init, obj_val = get_stotal(config, kmeans, decoded_centroids)

    free_centroids(decoded_centroids)
    free_kmeans(kmeans)

    print_results(stotal_lat_init.value, silouette_lat_init.val, obj_val.value)

    rows.append([model, config['model'].decode(), stotal_lat_init.value, obj_val.value])

    del silouette_lat_init

def run_kmeans_classic(model):
    normalized_dataset, encoded_dataset = model_to_files[model][0], model_to_files[model][2]

    config = {
        'model': b'CLASSIC',
        'vals': [],
        'dataset': normalized_dataset,
        'encoded_dataset': encoded_dataset,
    }
    
    compute_silhouette(model, config)

def run_kmeans_lsh(model):
    normalized_dataset, encoded_dataset = model_to_files[model][0], model_to_files[model][2]

    config = {
        'model': b'LSH',
        'vals': best_params_lsh[:-1],
        'window': best_params_lsh[-1],
        'dataset': normalized_dataset,
        'encoded_dataset': encoded_dataset,
    }

    compute_silhouette(model, config)

def run_kmeans_cube(model):
    normalized_dataset, encoded_dataset = model_to_files[model][0], model_to_files[model][2]

    config = {
        'model': b'CUBE',
        'vals': best_params_hypercube[:-1],
        'window': best_params_hypercube[-1],
        'dataset': normalized_dataset,
        'encoded_dataset': encoded_dataset,
    }

    compute_silhouette(model, config)

The following cell runs 4 threads in parallel at a time. When one finishes, next one is started. There is no need for locking as Python's code is always executed in a single thread because of GIL (Global Interpreter Lock). C++ code is executed in its own thread (and CPU for our case) and is not affected by GIL as it is released from ctypes module.

In [6]:
pool = ThreadPool(processes=4)

for model in models:
    pool.apply_async(run_kmeans_classic, (model,))
    pool.apply_async(run_kmeans_lsh, (model,))
    pool.apply_async(run_kmeans_cube, (model,))

pool.close()
pool.join()

10 inner and 2 outer loops
24 iterations
30 inner and 6 outer loops
1/1 [==============================] - 0s 258ms/step
91 iterations
1/1 [==============================] - 0s 187ms/step
Total silhouette: 0.1678379166417307
Silhouette per cluster: [0.1060730308587816, 0.10807911364439338, 0.13311019029723678, 0.1157627718139762, 0.08952708691193609, 0.2118840961494629, 0.20319515712850017, 0.16933775091530998, 0.34064295200678124, 0.12284454329344081]
Objective function: 5036991.70427464
-------------------------
35 inner and 7 outer loops
1/1 [==============================] - 0s 200ms/step
Total silhouette: 0.20480779239380056
Silhouette per cluster: [0.08962839198545193, 0.14377154578902707, 0.2600736153067117, 0.15074779252392206, 0.19801114929199118, 0.21029208362003332, 0.37521620532864053, 0.25900806440398444, 0.14562967303343155, 0.15022343205886346]
Objective function: 5159906.552967919
-------------------------
12 inner and 2 outer loops
1/1 [==============================] 

/root/miniconda3/envs/tf/lib/python3.10/site-packages/tensorflow/python/framework/constant_op.py:98: RuntimeWarning: overflow encountered in cast
  return ops.EagerTensor(value, ctx.device_name, dtype)


Total silhouette: 0.21104515999644635
Silhouette per cluster: [0.02533801743826257, 0.06676296903701726, 0.13632556042275853, 0.1919420112262875, 0.1696075733408128, 0.1137626504842817, 0.16436973263064686, 0.2540766319994949, 0.04139286189215293, 0.3029076423870727]
Objective function: 6906516.451897216
-------------------------
36 iterations
1/1 [==============================] - 0s 200ms/step
Total silhouette: 0.07614628784242047
Silhouette per cluster: [0.23697078368725732, -0.0234821765395499, 0.0458553787549225, 0.09320793602570324, 0.10329507306918315, 0.0063741316941190946, 0.08249866021814908, 0.08251787964423359, 0.09507357154866393, 0.029888619781089257]
Objective function: 4300464.594653758
-------------------------
20 inner and 4 outer loops
1/1 [==============================] - 0s 152ms/step
Total silhouette: 0.13904950084443532
Silhouette per cluster: [0.028334935667117565, 0.14276855623763435, 0.04637871915582884, 0.08125732003087688, 0.16409397465099787, 0.07918019078

In [7]:
col_models, col_methods, col_stotal_lat_init, col_obj_func = [], [], [], []

for row in rows:
    model, method, stotal_lat_init, obj_func = row

    col_models.append(model)
    col_methods.append(method)
    col_stotal_lat_init.append(stotal_lat_init)
    col_obj_func.append(obj_func)

col_dict = {'model': col_models, 'method': col_methods, 'silhouette total (latent to initial)': col_stotal_lat_init, 'objective function': col_obj_func}

df = pandas.DataFrame(data=col_dict)
df

,model,method,silhouette total (latent to initial),objective function
0,model_conv_46.keras,CLASSIC,0.167838,5.036992e+06
1,model_conv_12.keras,CLASSIC,0.204808,5.159907e+06
2,model_conv_46.keras,CUBE,0.163701,6.224723e+06
3,model_conv_46.keras,LSH,0.125937,6.842360e+06
4,model_conv_12.keras,CUBE,0.159740,6.674670e+06
5,model_conv_12.keras,LSH,0.211045,6.906516e+06
6,model_dense_43.keras,CLASSIC,0.076146,4.300465e+06
7,model_dense_43.keras,CUBE,0.139050,NaN
8,model_conv_19.keras,CLASSIC,0.150918,5.162240e+06
9,model_dense_43.keras,LSH,0.059264,5.449625e+06


# Results

## Best K-Means variation between initial and latent spaces

| model | latent dimension | best (total silhouette latent $\rightarrow$ initial) | best (objective function)
|:-----:|:----------------:|:-----------------------:|:-------------------------:|
| `model_conv_12.keras`    | $25$ | LSH     | CLASSIC |
| `model_conv_19.keras`    | $32$ | CLASSIC | CLASSIC |
| `model_conv_46.keras`    | $18$ | LSH     | CLASSIC |
| `model_dense_1.keras`    | $46$ | LSH     | CLASSIC |
| `model_dense_26.keras`   | $23$ | LSH     | CLASSIC |
| `model_dense_43.keras`   | $33$ | LSH     | CLASSIC |

## Worst K-Means variation between initial and latent spaces

| model | latent dimension | worst (total silhouette latent $\rightarrow$ initial) | worst (objective function)
|:-----:|:----------------:|:-----------------------:|:-------------------------:|
| `model_conv_12.keras`    | $25$ | CUBE | LSH |
| `model_conv_19.keras`    | $32$ | CUBE | LSH |
| `model_conv_46.keras`    | $18$ | LSH  | LSH |
| `model_dense_1.keras`    | $46$ | CUBE | CUBE |
| `model_dense_26.keras`   | $23$ | CUBE | CUBE |
| `model_dense_43.keras`   | $33$ | CUBE | CUBE |

# Conclusions

The Classic K-Means algorithm yields the best objective function values.

The Reverse LSH and Hypercube variations of the K-Means algorithm yields the worst objective function values in latent spaces of lower and higher dimension respectively.

The Reverse Hypercube variation almost always leads to the worst total silhouette value between the initial and latent space.
On the other hand, the Reverse LSH variation is the one that has the best value for this metric in most cases.

The fact that the objective function values are very close for all algorithms and models in the latent space (order of magnitude $10^6$) could mean that they probably converge to a similar local minimum. In contrast to the calculation of the objective function in the initial space (only for the Classic algorithm), objective function values in latent space are about $2-4$ times higher for all variations. We cannot know easily if this similar value is near the global minimum or not as clustering is an NP-hard problem (finding the optimal way to group a set of objects into clusters based on certain criteria, such as maximizing similarity within clusters and minimizing similarity between clusters; in other words minimizing the defined objective function).

Compared to the nearest neighbor search, projection to lower dimension spaces at clustering K-Means methods seems to have impressive results. The convergence to a local minimum is much faster, but also we can see that the quality of the clustering is the same or even better than in the initial space. This is probably due to the fact that the generalized representation of a digit in the latent space is more suitable for "grouping" digits of the same class together (thanks to the model's loss!).